# Rough

In [3]:
%load_ext autoreload
%autoreload 2

# %matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd

import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
import optuna

from sklearn.metrics import mean_squared_error

import warnings

# from poly2.utils import (
#     get_dispersal_kernel, trait_vec, load_train_test_data,
#     get_best_model, get_model_cv_score, HyperparamsObj,
#     get_best_params, 
# )
# from cluster.hyper import train_test_scores, run_optuna
# from poly2.config import Config

/Users/nicktaylor/opt/miniconda3/envs/poly2/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


ModuleNotFoundError: No module named 'poly2'